In [ ]:
!pip install --upgrade pip
!pip install backtrader yfinance pyfolio matplotlib pandas numpy


#Download crypto data (using yfinance)

In [ ]:
import yfinance as yf

# Downloading BTC-USD daily data for the last 1 year
data = yf.download('BTC-USD', start='2023-01-01', end='2024-01-01')

print(data.head())


<ipython-input-49-168098511>:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('BTC-USD', start='2023-01-01', end='2024-01-01')
[*********************100%***********************]  1 of 1 completed

Price              Close          High           Low          Open  \
Ticker           BTC-USD       BTC-USD       BTC-USD       BTC-USD   
Date                                                                 
2023-01-01  16625.080078  16630.439453  16521.234375  16547.914062   
2023-01-02  16688.470703  16759.343750  16572.228516  16625.509766   
2023-01-03  16679.857422  16760.447266  16622.371094  16688.847656   
2023-01-04  16863.238281  16964.585938  16667.763672  16680.205078   
2023-01-05  16836.736328  16884.021484  16790.283203  16863.472656   

Price            Volume  
Ticker          BTC-USD  
Date                     
2023-01-01   9244361700  
2023-01-02  12097775227  
2023-01-03  13903079207  
2023-01-04  18421743322  
2023-01-05  13692758566  


#Clean and rename data columns for Backtrader

In [ ]:
print(data.columns.tolist())

[('Close', 'BTC-USD'), ('High', 'BTC-USD'), ('Low', 'BTC-USD'), ('Open', 'BTC-USD'), ('Volume', 'BTC-USD')]


In [ ]:
data.rename(columns={
    'Open_BTC-USD': 'open',
    'High_BTC-USD': 'high',
    'Low_BTC-USD': 'low',
    'Close_BTC-USD': 'close',
    'Volume_BTC-USD': 'volume'
}, inplace=True)


In [ ]:
print(data.columns.tolist())

[('Close', 'BTC-USD'), ('High', 'BTC-USD'), ('Low', 'BTC-USD'), ('Open', 'BTC-USD'), ('Volume', 'BTC-USD')]


In [ ]:
# Flattening MultiIndex columns: from ('close', 'BTC-USD') → 'close'
data.columns = [col[0].lower() if isinstance(col, tuple) else col.lower() for col in data.columns]

# Replacing infs with NaNs and drop missing values
import numpy as np
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)

print("✅ Data cleaned and column names flattened.")


✅ Data cleaned and column names flattened.


# Defining The Strategy

###Moving Average Crossover Strategy

In [ ]:
import backtrader as bt

class SmaCross(bt.Strategy):
    params = (('fast', 20), ('slow', 50))

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.fast)
        sma2 = bt.ind.SMA(period=self.p.slow)
        self.crossover = bt.ind.CrossOver(sma1, sma2)

    def next(self):
        if not self.position and self.crossover > 0:
            self.buy()
        elif self.position and self.crossover < 0:
            self.close()


##Preparing Data Feed

In [ ]:
data_bt = bt.feeds.PandasData(dataname=data)


##Setting Up Backtest Engine

In [ ]:
cerebro = bt.Cerebro()
cerebro.addstrategy(SmaCross)
cerebro.adddata(data_bt)
cerebro.broker.setcash(10000)


##Running & Plotting

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
results = cerebro.run()
cerebro.plot()



[[<Figure size 640x480 with 40 Axes>]]

#Analyze the Strategy Performance with PyFolio

Generate tearsheets (like hedge funds do)

Display Sharpe ratio, drawdowns, returns, volatility

In [ ]:
#import and installing pyfolio

!pip install pyfolio
import pyfolio as pf


#Extracting Return from Backtrader

In [ ]:
class PyFolioAnalyzer(bt.Analyzer):
    def __init__(self):
        self.rets = []

    def notify_trade(self, trade):
        if trade.isclosed:
            pnl = trade.pnl / trade.price
            self.rets.append(pnl)

    def get_analysis(self):
        return self.rets


In [ ]:
def fix_col(col):
    # Removing underscores between letters
    no_underscores = col.replace('_', '')
    # Inserting underscore before 'btc-usd' part
    fixed = no_underscores.replace('btc-usd', '_btc-usd')
    return fixed

btc_df.columns = [fix_col(col) for col in btc_df.columns]

print(btc_df.columns.tolist())


['close_btc-usd', 'high_btc-usd', 'low_btc-usd', 'open_btc-usd', 'volume_btc-usd']


In [ ]:
class PandasData(bt.feeds.PandasData):
    params = (
        ('datetime', None),
        ('open', 'open_btc-usd'),
        ('high', 'high_btc-usd'),
        ('low', 'low_btc-usd'),
        ('close', 'close_btc-usd'),
        ('volume', 'volume_btc-usd'),
        ('openinterest', None),
    )


In [ ]:

cerebro = bt.Cerebro()
data = PandasData(dataname=btc_df)
cerebro.adddata(data)
cerebro.broker.setcash(10000)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
results = cerebro.run()
